In [1]:
import requests
import json
import os
import urllib.parse


def clean_battle_data(battle_data):
    # Define keys to retain
    desired_keys = {'name', 'level', 'elixirCost', 'rarity'}
    
    def filter_card_data(card):
        # Filter each card's data to retain only desired keys
        return {key: card[key] for key in card if key in desired_keys}

    def process_team_or_opponent(team_or_opponent):
        # Process each team or opponent's cards
        team_or_opponent['cards'] = [filter_card_data(card) for card in team_or_opponent['cards']]
        return team_or_opponent

    # Process the entire battle data
    for battle in battle_data:
        battle['team'] = [process_team_or_opponent(member) for member in battle['team']]
        battle['opponent'] = [process_team_or_opponent(member) for member in battle['opponent']]
    
    return battle_data

# Example usage with your JSON data
import json

# Assuming `battle_data` is your original JSON response loaded as a Python object
#cleaned_data = clean_battle_data(battle_data)

# Pretty-print the cleaned data
#print(json.dumps(cleaned_data, indent=2))

def get_player_data(player_tag, api_key):
    # URL-encode the player tag
    #encoded_tag = urllib.parse.quote(player_tag)
    
    # Define the API endpoint
    url = f'https://api.clashroyale.com/v1/players/{player_tag}/battlelog'
    
    # Set up the headers with the API key
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }
    
    # Make the GET request to the API
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error: {response.status_code}')
        return None

if __name__ == '__main__':
    # 1. Read the API key from the 'data/clash-royale-api-key.txt' file
    key_file_path = os.path.join('../data/clash-royale-api-key.txt')
    with open(key_file_path, 'r') as file:
        api_key = file.read().strip()
    
    # 2. Specify the player tag (URL-encoded # is often %23 in Clash Royale)
    player_tag = '%232LGY9G'  # Replace with the actual player tag, e.g. #2LGY9G -> %232LGY9G
    
    # 3. Call the API
    player_data = get_player_data(player_tag, api_key)
    if player_data:
        # 4. Clean the data
        cleaned_data = clean_battle_data(player_data)

        # 5. Save cleaned data to 'outputCR.json' in the 'data' folder
        output_path = os.path.join('../data/outputCR.json')
        with open(output_path, 'w', encoding='utf-16') as out_file:
            json.dump(cleaned_data, out_file, indent=2)
        
        # 6. Print the JSON file contents
        print(f"Saved cleaned data to: {output_path}\n")
        print("Printing contents of outputCR.json:\n")
        with open(output_path, 'r', encoding='utf-16') as in_file:
            file_data = json.load(in_file)
            print(json.dumps(file_data, indent=2))
    else:
        print("No data received from API. Please check your player tag or API key.")

Saved cleaned data to: ../data/outputCR.json

Printing contents of outputCR.json:

[
  {
    "type": "pathOfLegend",
    "battleTime": "20241228T135508.000Z",
    "isLadderTournament": false,
    "arena": {
      "id": 54000098,
      "name": "Legendary Arena"
    },
    "gameMode": {
      "id": 72000464,
      "name": "Ranked1v1_NewArena2"
    },
    "deckSelection": "collection",
    "team": [
      {
        "tag": "#2LGY9G",
        "name": "Diwel26",
        "crowns": 1,
        "kingTowerHitPoints": 5721,
        "princessTowersHitPoints": null,
        "clan": {
          "tag": "#YULCL0YQ",
          "name": "ITA",
          "badgeId": 16000081
        },
        "cards": [
          {
            "name": "Royal Giant",
            "level": 13,
            "rarity": "common",
            "elixirCost": 6
          },
          {
            "name": "Firecracker",
            "level": 13,
            "rarity": "common",
            "elixirCost": 3
          },
          {
      

In [2]:
import pandas as pd
import json

# Dictionary mapping card names to IDs
cards_dict = {
    "Archers": 1,
    "Archer Queen": 2,
    "Arrows": 3,
    "Baby Dragon": 4,
    "Balloon": 5,
    "Bandit": 6,
    "Barbarians": 7,
    "Barbarian Barrel": 8,
    "Barbarian Hut": 9,
    "Bats": 10,
    "Battle Healer": 11,
    "Battle Ram": 12,
    "Bomber": 13,
    "Bomb Tower": 14,
    "Bowler": 15,
    "Bush Goblins": 16,
    "Cannon": 17,
    "Cannon Cart": 18,
    "Cursed Hog": 19,
    "Dark Prince": 20,
    "Dart Goblin": 21,
    "Defensive Buildings": 22,
    "Earthquake": 23,
    "Electro Dragon": 24,
    "Electro Giant": 25,
    "Electro Spirit": 26,
    "Electro Wizard": 27,
    "Elixir Blob": 28,
    "Elixir Collector": 29,
    "Elixir Golem": 30,
    "Elixir Golemite": 31,
    "Executioner": 32,
    "Fireball": 33,
    "Fire Spirit": 34,
    "Firecracker": 35,
    "Fisherman": 36,
    "Flying Machine": 37,
    "Freeze": 38,
    "Furnace": 39,
    "Giant": 40,
    "Giant Skeleton": 41,
    "Giant Snowball": 42,
    "Goblin Barrel": 43,
    "Goblin Brawler": 44,
    "Goblin Cage": 45,
    "Goblin Curse": 46,
    "Goblin Demolisher": 47,
    "Goblin Drill": 48,
    "Goblin Gang": 49,
    "Goblin Giant": 50,
    "Goblin Hut": 51,
    "Goblin Machine": 52,
    "Goblinstein": 53,
    "Goblins": 54,
    "Golden Knight": 55,
    "Golem": 56,
    "Golemite": 57,
    "Graveyard": 58,
    "Guardienne": 59,
    "Guards": 60,
    "Heal Spirit": 61,
    "Hog Rider": 62,
    "Hunter": 63,
    "Ice Golem": 64,
    "Ice Spirit": 65,
    "Ice Wizard": 66,
    "Inferno Dragon": 67,
    "Inferno Tower": 68,
    "Knight": 69,
    "Lava Hound": 70,
    "Lava Pup": 71,
    "Lightning": 72,
    "Little Prince": 73,
    "Log": 74,
    "Lumberjack": 75,
    "Magic Archer": 76,
    "Mega Knight": 77,
    "Mega Minion": 78,
    "Mighty Miner": 79,
    "Miner": 80,
    "Mini P.E.K.K.A.": 81,
    "Minion Horde": 82,
    "Minions": 83,
    "Monk": 84,
    "Monster": 85,
    "Mortar": 86,
    "Mother Witch": 87,
    "Musketeer": 88,
    "Night Witch": 89,
    "P.E.K.K.A.": 90,
    "Phoenix": 91,
    "Phoenix Egg": 92,
    "Poison": 93,
    "Prince": 94,
    "Princess": 95,
    "Ram Rider": 96,
    "Rage": 97,
    "Rascal Boy": 98,
    "Rascal Girl": 99,
    "Reborn Phoenix": 100,
    "Rocket": 101,
    "Royal Delivery": 102,
    "Royal Ghost": 103,
    "Royal Giant": 104,
    "Royal Hogs": 105,
    "Royal Recruits": 106,
    "Skeleton Army": 107,
    "Skeleton Barrel": 108,
    "Skeleton Dragons": 109,
    "Skeleton King": 110,
    "Skeletons": 111,
    "Sparky": 112,
    "Spear Goblins": 113,
    "Spawners": 114,
    "Suspicious Bush": 115,
    "Tesla": 116,
    "The Log": 117,
    "Three Musketeers": 118,
    "Tombstone": 119,
    "Tornado": 120,
    "Valkyrie": 121,
    "Void": 122,
    "Wall Breakers": 123,
    "Witch": 124,
    "Wizard": 125,
    "X-Bow": 126,
    "Zap": 127,
    "Zappies": 128
}

# Assuming the provided JSON is saved in 'outputCR.json'
try:
    with open('../data/outputCR.json', 'r', encoding='utf-16') as f:
        battle_data = json.load(f)  # Load the JSON data
except FileNotFoundError:
    print("File not found. Ensure 'outputCR.json' exists in the directory.")
    battle_data = None
except json.JSONDecodeError as e:
    print(f"Invalid JSON format: {e}")
    battle_data = None

if battle_data:
    # Function to process a single player's data
    def process_player_data(player, is_team=True):
        deck = [cards_dict.get(card['name'], -1) for card in player['cards']]  # Convert names to IDs
        support_cards = [cards_dict.get(card['name'], -1) for card in player.get('supportCards', [])]
        trophy_change = player.get('trophyChange', 0) * (1 if is_team else -1)
        return {
            'Player Name': player['name'],
            **{f'Card {i+1}': card for i, card in enumerate(deck)},  # One column per card
            #'Support Cards': ', '.join(map(str, support_cards)),
            #'Trophy Change': trophy_change,
            #'Crowns': player['crowns']
        }

    # Collect all matches
    all_matches = []

    for match in battle_data:
        team = match['team'][0]
        opponent = match['opponent'][0]
        team_data = process_player_data(team, is_team=True)
        opponent_data = process_player_data(opponent, is_team=False)

        # Add Win or Lose based on crowns
        if team['crowns'] > opponent['crowns']:
            team_data['Result'] = 'WIN'
            opponent_data['Result'] = 'LOSE'
        elif team['crowns'] < opponent['crowns']:
            team_data['Result'] = 'LOSE'
            opponent_data['Result'] = 'WIN'
        else:
            team_data['Result'] = 'DRAW'
            opponent_data['Result'] = 'DRAW'

        all_matches.append(team_data)
        all_matches.append(opponent_data)

    # Create the DataFrame
    df = pd.DataFrame(all_matches)

    # Display the DataFrame
    print(df)
else:
    print("No battle data available.")


        Player Name  Card 1  Card 2  Card 3  Card 4  Card 5  Card 6  Card 7  \
0           Diwel26     104      35       8      36      33     110      87   
1       killergamer      77      10       6       3       2      94     123   
2           Diwel26     104      35       8      36      33     110      87   
3             J.MR✓      35      77      30      33      14      62     124   
4           Diwel26      30      11     127      97      67      24      -1   
5             원주민준우      86      24      49       8     120      48      66   
6           Diwel26      30      11     127      97      67      24      -1   
7               갓대현      77      35      94      95      33      36       4   
8           Diwel26      30      11     127      97      67      24      -1   
9     papillon d'or      25      35      83      33      84      95     124   
10          Diwel26      30      11     127      97      67      24      -1   
11            felix      -1      27      21     117 

In [5]:
import os
import hopsworks
import pandas as pd

# 1. Read your Hopsworks API Key from a file or environment variable
#    For example, if stored in 'hopsworks-api-key.txt'
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key

# 3. Log in to your Hopsworks project
project = hopsworks.login()  # or hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

# 4. Suppose we already have a DataFrame named df (e.g., from the previous step)
#    that has columns like: "player_name", "deck", "crowns", "result", "battle_time", etc.
#    Make sure df is defined before this script or above in the same script:
# df = ...

# 5. Create (or get) a feature group
cr_feature_group = fs.get_or_create_feature_group(
    name="clash_royale_features",
    version=1,
    description="Clash Royale cleaned match features",
    primary_key=["player_name"],  # columns that uniquely identify each row
)

# 6. Insert data into the feature group
cr_feature_group.insert(df)

print("DataFrame inserted into Hopsworks Feature Store successfully!")


2025-01-03 18:03:58,087 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-03 18:03:58,095 INFO: Initializing external client
2025-01-03 18:03:58,096 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-03 18:03:59,399 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1175700
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1175700/fs/1166403/fg/1393497


KafkaException: KafkaError{code=_TRANSPORT,val=-195,str="Failed to get metadata: Local: Broker transport failure"}